In [1]:
"""
To-do list:
Actually parse stuff
Simplify party names when there are more (Green, Reform)
Clean up judicial shit
"""

'\nTo-do list:\nActually parse stuff\nSimplify party names when there are more (Green, Reform)\nClean up judicial shit\n'

In [2]:
import csv
import glob
import time
import datetime

In [3]:
primary = True
datadir = "snapshots/"
racedelim = " -- "    # E.g., "U.S. Senator -- Rep."
folders = sorted(list(glob.glob(datadir + "*")), reverse=True)    # Find the latest time-stamped folder
folder = folders[0] + "/"
if len(glob.glob(folder + "*")) != 4:   # 3 file native file types and a done file. If not 4 files, it's not done
    time.sleep(10)   # 10 seconds to beat a race condition
    if len(glob.glob(folders[0] + "/*")) != 4:
        print(quit)

In [4]:
def get_timestamp():
    global folder
    rawtimestamp = folder.split("-")[1].replace("/", "")
    hour = int(rawtimestamp[0:2])
    if hour > 12:
        hour = hour -12
    if hour == 0:
        hour = 12
    hour = str(hour)
    timestamp = hour + ":" + rawtimestamp[2:4]
    return(timestamp)

In [5]:
def cleanrow(row):
    global primary
    global racedelim
    for item in ("Precincts", "PrecinctsReporting", "CanVotes"):
        row[item] = int(row[item])    # Turn into numbers
    partysubs = [
        ("Republican Party", "Rep."),
        ("Democratic Party", "Dem."),
        ("Non-Partisan", ""),
        ("Green Party", "Green"),
        ("Reform Party", "Reform")
    ]
    row['ShortParty'] = row['PartyName']
    for partysub in partysubs:
        row['ShortParty'] = row['ShortParty'].replace(partysub[0], partysub[1])
    racesubs = [
        ("United States ", "U.S. "),
        ("Representative in Congress, District ", "U.S. Congress, District "),
        ("Circuit Judge, ", "Judge, "),
        ("State Representative, District ", "State Rep., District ")
    ]
    for racesub in racesubs:
        row['RaceName'] = row['RaceName'].replace(racesub[0], racesub[1])

    if not primary:
        row['FullRace'] = row['RaceName']
        row['Partisan'] = 0
    else:
        if len(row['ShortParty']) == 0:
            row['FullRace'] = row['RaceName']
            row['Partisan'] = 0
        else:
            row['FullRace'] = row['RaceName'] + racedelim + row['ShortParty']
            row['Partisan'] = 1
    return(row)

In [6]:
with open(folder + "results.txt", "r") as f:    # Import the data and do some basic cleaning
    masterlist = []
    for row in csv.DictReader(f, delimiter="\t"):
        masterlist.append(cleanrow(row))

In [7]:
with open("recastreport.csv", "w", newline="") as f:
    headers = row.keys()
    writer = csv.writer(f)
    writer.writerow(headers)
    for row in masterlist:
        line = []
        for item in headers:
            line.append(str(row[item]))
        writer.writerow(line)